<a href="https://www.kaggle.com/code/elcientfico/gbt-for-salary-estimation?scriptVersionId=147908817" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory



# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
def convert_categorical_to_numeric(df):
    # Separate columns into numeric and non-numeric (categorical)
    numeric_cols = df.select_dtypes(include=['number']).columns
    categorical_cols = df.select_dtypes(exclude=['number']).columns

    # Preserve the numeric columns
    numeric_df = df[numeric_cols]

    # Convert categorical columns to numerical using one-hot encoding
    categorical_df = pd.get_dummies(df[categorical_cols], drop_first=True)

    # Combine the numeric and one-hot encoded categorical data
    result_df = pd.concat([numeric_df, categorical_df], axis=1)

    return result_df

In [ ]:
df = pd.read_csv("/kaggle/input/data-jobs-salaries/salaries.csv")
df.pop("salary")
y = df.pop("salary_in_usd").to_frame()
x = df
x = convert_categorical_to_numeric(x)


In [ ]:
df.info()

In [ ]:
df = pd.read_csv("/kaggle/input/data-jobs-salaries/salaries.csv")
pd.set_option('display.max_colwidth', None) # for pandas >= 1.x
pd.set_option('display.max_columns', None)
corr_simple = x.corr().abs()
np.fill_diagonal(corr_simple.values, 0)
corr_simple.max()

In [ ]:
x.info()

In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor
from xgboost import XGBRegressor
my_model = XGBRegressor()
from sklearn.model_selection import train_test_split

Sppkit and train

In [ ]:
def z_statistic(y1,y2):
    numerator = abs(y1.mean() - y2.mean())
    
    sigma1 = (y1.std()**2)/np.sqrt(len(y1))
    sigma2 = (y2.std()**2)/np.sqrt(len(y2))
                    
    denominator = np.sqrt(sigma1 - sigma2)
                    
    return numerator/denominator

In [ ]:
z1 = np.inf
z2 = np.inf
current_state = 0
best_state = current_state
best_sum = np.inf
for i in range(10000):
    x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = .2,shuffle = True,random_state = current_state)
    z1 = z_statistic(y_train.values,y.values)
    z2 = z_statistic(y_test.values,y.values)
    current_state+=1
    if (best_sum)>(z1+z2):
        print(current_state,":",z1 + z2)
        best_state = current_state
        best_sum = z1 + z2
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = .2,shuffle = True,random_state = best_state)

In [ ]:
plt.hist(y,bins = 100)
plt.hist(y_train,bins = 100)
plt.hist(y_test,bins = 100)

In [ ]:
rfr = HistGradientBoostingRegressor(max_iter =9999999999999,verbose = 0,
                                    early_stopping = True, max_leaf_nodes =7, 
                                    learning_rate = .001,n_iter_no_change = 100,min_samples_leaf = 1)
rfr.fit(x_train,y_train)

print(rfr.score(x_train,y_train),rfr.score(x_test,y_test))

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error

In [ ]:
print(mean_absolute_percentage_error(y_test,rfr.predict(x_test)))
print(mean_absolute_percentage_error(y_train,rfr.predict(x_train)))

In [ ]:
plt.hist(rfr.predict(x_train),100)
plt.hist(rfr.predict(x_test),100)

In [ ]:
err1 = abs(rfr.predict(x_train) - y_train.values.squeeze())
err2 = abs(rfr.predict(x_test) - y_test.values.squeeze())
plt.hist(err1,100)
plt.hist(err2,100)

In [ ]:
plt.plot(np.sort(np.abs(y_test.squeeze().values - rfr.predict(x_test))))

Lets check for the USA BIAS

In [ ]:
print(mean_absolute_percentage_error(y_test[x_test.company_location_US == True],rfr.predict(x_test[x_test.company_location_US == True])))
